In [45]:
import json
from collections import defaultdict

def get_response(fname):
    with open(fname, 'r') as f:
        data = f.read()

    response = json.loads(data)

    surv_resp = defaultdict(dict)
    pass_resp = defaultdict(str)
    time_resp = defaultdict(str)

    for k, v in response.items():
        if 'response' in k:
            temp = k.split("_")
            pid = temp[0].strip('p')
            v = json.loads(v)
            surv_resp[pid].update(v)
        elif 'time' in k:
            temp = k.split("_")
            pid = temp[0]
            time_resp[pid] = float(v)
        elif 'password' in k:
            pass_resp[k] = v

    return (pass_resp, time_resp, surv_resp)

In [28]:
page_map = {
    '0': 'startstudy',
    '1': 'fpass',
    '2': 'fcog',
    '3': 'fpasssurvey',
    '4': 'fpassrecall',
    '5': 'pagegif',
    '6': 'desinint',
    '7': 'quest',
    '8': 'csquest',
    '9': 'spass',
    '10': 'scog',
    '11': 'spasssurvey',
    '12': 'spassrecall',
    '13': 'demo',
    '14': 'thanks',
}

In [46]:
import os
fpath = './responses'
password_response = []
time_response = []
survey_response = []
for i, fname in enumerate(os.listdir(fpath)):
    pss, tim, sur = get_response(os.path.join(fpath, fname))
    password_response.append(pss)
    time_response.append(tim)
    survey_response.append(sur)

In [47]:
import pandas as pd
pd.DataFrame(password_response)

,password1,password2_recall1,password2,password1_recall1,password2_recall2,password1_recall3,password1_recall2
0,21,aw,as,34,aw,NaN,NaN
1,ddfsfds,NaN,asd,asda,NaN,ddsda,dasdas


In [48]:
page_time = pd.DataFrame(time_response)
page_time

,12,1,6,7,9,0,10,4,2,3,5
0,12.289,7.116,17.147,19.531,38.820,5.809,2.694,7.095,3.239,15.487,3.054
1,3.141,12.029,4.320,19.845,5.639,3.295,6.824,6.747,3.496,10.457,2.276


In [94]:
# top 5 pages with higher time
top5 = pd.DataFrame(columns=['page', 'time'])
for i, data in page_time.mean().sort_values(ascending=False).head(5).items():
    top5 = top5.append(pd.DataFrame([[page_map.get(i), data]], columns=top5.columns), ignore_index=True)

top5

,page,time
0,spass,22.2295
1,quest,19.6880
2,fpasssurvey,12.9720
3,desinint,10.7335
4,fpass,9.5725


In [34]:
pd.DataFrame(survey_response)

,7,3,8,13,11
0,"{'1-19': '5', '1-20': '5', '1-21': '5', '1-22'...","{'1-1': '3', '2-1': '4', '2-2': '4', '2-3': '4...","{'1-1': '7', '1-2': '7', '1-3': '7', '1-4': '7...","{'1-1': 'Transgender/Trans woman', '2-1': '35-...","{'1-1': '7', '2-1': '7', '2-2': '7', '2-3': '7..."
1,"{'1-19': '3', '1-20': '3', '1-21': '4', '1-22'...","{'1-1': '3', '2-1': '4', '2-2': '5', '2-3': '6...","{'1-1': '3', '1-2': '4', '1-3': '4', '1-4': '5...","{'1-1': 'Man', '2-1': '40-44 years old', '3-1'...","{'1-1': '4', '2-1': '4', '2-2': '5', '2-3': '5..."
